https://www.youtube.com/watch?v=KYFfMymTspE

In [1]:
from keras.applications.resnet import ResNet50, decode_predictions
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K

Using TensorFlow backend.


In [2]:
img_rows,img_cols = 224, 224

Resnet = ResNet50(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in Resnet.layers:
    layer.trainable=False

def addTopModelResnet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 11

FC_Head = addTopModelResnet(Resnet,num_classes)

model = Model(inputs = Resnet.input, outputs = FC_Head)

print(model.summary())

/home/lab28/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [3]:
train_data_dir = 'train'
validation_data_dir = 'test'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'finger_sign_Resnet50.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 10
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 5262 images belonging to 11 classes.
Found 5262 images belonging to 11 classes.
Epoch 1/10
37/37 [==============================] - 20s 550ms/step - loss: 2.0074 - accuracy: 0.2905 - val_loss: 2.5787 - val_accuracy: 0.0764

Epoch 00001: val_loss improved from inf to 2.57875, saving model to finger_sign_Resnet50.h5
Epoch 2/10
37/37 [==============================] - 14s 371ms/step - loss: 1.1369 - accuracy: 0.6321 - val_loss: 2.5291 - val_accuracy: 0.0694

Epoch 00002: val_loss improved from 2.57875 to 2.52913, saving model to finger_sign_Resnet50.h5
Epoch 3/10
37/37 [==============================] - 16s 421ms/step - loss: 0.5977 - accuracy: 0.7931 - val_loss: 2.5885 - val_accuracy: 0.0799

Epoch 00003: val_loss did not improve from 2.52913
Epoch 4/10
37/37 [==============================] - 16s 430ms/step - loss: 0.4647 - accuracy: 0.8505 - val_loss: 3.1764 - val_accuracy: 0.0521

Epoch 00004: val_loss did not improve from 2.52913
Epoch 5/10
37/37 [==============================